<a href="https://colab.research.google.com/github/EduBrazGomes/ProjetoIntegrador/blob/main/Quest%C3%A3o8.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Etapa 1: Importar bibliotecas e carregar os dados
import pandas as pd
import zipfile
import urllib.request

# URL do dataset MovieLens 1M
url = 'http://files.grouplens.org/datasets/movielens/ml-1m.zip'

# Baixar o arquivo
print("Baixando o dataset MovieLens...")
urllib.request.urlretrieve(url, 'movielens.zip')
print("Download concluído.")

# Extrair o arquivo zip
print("Extraindo arquivos...")
with zipfile.ZipFile('movielens.zip', 'r') as zip_ref:
    zip_ref.extractall()
print("Extração concluída.")

# Carregar os datasets de avaliações e filmes usando pandas
# O dataset ml-1m usa '::' como separador
ratings = pd.read_csv('ml-1m/ratings.dat', sep='::', engine='python', header=None, names=['userId', 'movieId', 'rating', 'timestamp'])
movies = pd.read_csv('ml-1m/movies.dat', sep='::', engine='python', header=None, names=['movieId', 'title', 'genres'], encoding='latin-1')


# Exibir as primeiras linhas para verificar
print("\n--- Dados de Avaliações (ratings) ---")
print(ratings.head())

print("\n--- Dados dos Filmes (movies) ---")
print(movies.head())


Baixando o dataset MovieLens...
Download concluído.
Extraindo arquivos...
Extração concluída.

--- Dados de Avaliações (ratings) ---
   userId  movieId  rating  timestamp
0       1     1193       5  978300760
1       1      661       3  978302109
2       1      914       3  978301968
3       1     3408       4  978300275
4       1     2355       5  978824291

--- Dados dos Filmes (movies) ---
   movieId                               title                        genres
0        1                    Toy Story (1995)   Animation|Children's|Comedy
1        2                      Jumanji (1995)  Adventure|Children's|Fantasy
2        3             Grumpier Old Men (1995)                Comedy|Romance
3        4            Waiting to Exhale (1995)                  Comedy|Drama
4        5  Father of the Bride Part II (1995)                        Comedy


In [ ]:
# Etapa 2: Preparação e Divisão dos Dados

from sklearn.model_selection import train_test_split
import numpy as np

# Para facilitar, vamos trabalhar com um número menor de usuários e filmes
# para que os cálculos sejam mais rápidos.
# Vamos pegar apenas os usuários que fizeram mais de 100 avaliações.
user_counts = ratings['userId'].value_counts()
active_users = user_counts[user_counts > 100].index
ratings_subset = ratings[ratings['userId'].isin(active_users)]

# Agora, vamos juntar os dados (merge)
data = pd.merge(ratings_subset, movies, on='movieId')

# Exibir as primeiras linhas dos dados combinados
print("--- Dados Combinados (Avaliações + Títulos dos Filmes) ---")
print(data.head())


# Dividir os dados em conjunto de treino (80%) e teste (20%)
train_data, test_data = train_test_split(data, test_size=0.2, random_state=42)

print(f"\nNúmero de avaliações para treino: {len(train_data)}")
print(f"Número de avaliações para teste: {len(test_data)}")


# Criar a matriz usuário-item a partir dos dados de TREINO
# Linhas: usuários, Colunas: filmes, Valores: notas
print("\n--- Matriz Usuário-Item (amostra) ---")
user_item_matrix = train_data.pivot_table(index='userId', columns='title', values='rating')

# Preencher valores ausentes (NaN) com 0
# NaN significa que o usuário não avaliou o filme
user_item_matrix.fillna(0, inplace=True)

# Exibir as primeiras 5 linhas e 5 colunas da matriz
print(user_item_matrix.iloc[:5, :5])

--- Dados Combinados (Avaliações + Títulos dos Filmes) ---
   userId  movieId  rating  timestamp  \
0       2     1357       5  978298709   
1       2     3068       4  978299000   
2       2     1537       4  978299620   
3       2      647       3  978299351   
4       2     2194       4  978299297   

                                      title              genres  
0                              Shine (1996)       Drama|Romance  
1                       Verdict, The (1982)               Drama  
2  Shall We Dance? (Shall We Dansu?) (1996)              Comedy  
3                 Courage Under Fire (1996)           Drama|War  
4                  Untouchables, The (1987)  Action|Crime|Drama  

Número de avaliações para treino: 674961
Número de avaliações para teste: 168741

--- Matriz Usuário-Item (amostra) ---
title   $1,000,000 Duck (1971)  'Night Mother (1986)  \
userId                                                 
2                          0.0                   0.0   
5        

In [ ]:
# Etapa 3: Filtragem Colaborativa Baseada em Usuários

from sklearn.metrics.pairwise import cosine_similarity

# Calcular a similaridade de cosseno entre os usuários
# A matriz user_item_matrix tem usuários nas linhas e filmes nas colunas
print("Calculando a similaridade entre usuários...")
user_similarity = cosine_similarity(user_item_matrix)

# Converter o resultado em um DataFrame do pandas para melhor visualização
user_similarity_df = pd.DataFrame(user_similarity, index=user_item_matrix.index, columns=user_item_matrix.index)

print("Matriz de Similaridade entre Usuários (amostra):")
print(user_similarity_df.iloc[:5, :5])


# --- Função para fazer recomendações ---
def get_user_recommendations(user_id, num_recommendations=5, k=10):
    """
    Gera recomendações para um usuário com base nos 'k' usuários mais similares.
    """
    # Encontrar os k usuários mais similares (excluindo o próprio usuário)
    similar_users = user_similarity_df[user_id].sort_values(ascending=False)[1:k+1]
    similar_user_ids = similar_users.index

    # Pegar as avaliações dos filmes feitas por esses usuários similares
    movies_rated_by_similar_users = user_item_matrix.loc[similar_user_ids]

    # Calcular a média das avaliações para cada filme (esta será nossa pontuação de recomendação)
    recommendation_scores = movies_rated_by_similar_users.mean(axis=0)

    # Filtrar para não recomendar filmes que o usuário já viu
    user_movies = user_item_matrix.loc[user_id]
    unseen_movies_scores = recommendation_scores[user_movies == 0]

    # Retornar os N filmes com as melhores pontuações
    return unseen_movies_scores.sort_values(ascending=False).head(num_recommendations)


# --- Testando o sistema de recomendação ---
# Escolha um ID de usuário que exista nos dados de treino para testar
# Vamos pegar um usuário do nosso user_item_matrix
test_user_id = user_item_matrix.index[10] # Pegando o 11º usuário da nossa matriz

print(f"\n--- Gerando 5 recomendações para o usuário ID: {test_user_id} ---")
recommendations = get_user_recommendations(test_user_id, num_recommendations=5, k=15)

print(recommendations)

Calculando a similaridade entre usuários...
Matriz de Similaridade entre Usuários (amostra):
userId        2         5         8         9         10
userId                                                  
2       1.000000  0.078203  0.156484  0.118966  0.197910
5       0.078203  1.000000  0.140488  0.226639  0.103401
8       0.156484  0.140488  1.000000  0.186526  0.123132
9       0.118966  0.226639  0.186526  1.000000  0.213133
10      0.197910  0.103401  0.123132  0.213133  1.000000

--- Gerando 5 recomendações para o usuário ID: 19 ---
title
Terminator, The (1984)                                4.200000
Austin Powers: International Man of Mystery (1997)    3.066667
Total Recall (1990)                                   3.000000
Rush Hour (1998)                                      3.000000
Independence Day (ID4) (1996)                         2.933333
dtype: float64


In [ ]:
# Etapa 4: Filtragem Colaborativa Baseada em Itens

# Para a filtragem baseada em itens, precisamos da similaridade entre os itens (filmes).
# Para isso, transpomos a matriz para que os filmes fiquem nas linhas.
item_user_matrix = user_item_matrix.T

print("Calculando a similaridade entre filmes...")
item_similarity = cosine_similarity(item_user_matrix)

# Converter em um DataFrame para melhor visualização
item_similarity_df = pd.DataFrame(item_similarity, index=item_user_matrix.index, columns=item_user_matrix.index)

print("Matriz de Similaridade entre Filmes (amostra):")
print(item_similarity_df.iloc[:5, :5])


# --- Função para fazer recomendações baseada em itens ---
def get_item_recommendations(user_id, num_recommendations=5):
    """
    Gera recomendações para um usuário com base na similaridade entre itens.
    """
    # Obter as avaliações do usuário
    user_ratings = user_item_matrix.loc[user_id]

    # Identificar os filmes que o usuário avaliou positivamente
    rated_movies = user_ratings[user_ratings > 0].index

    # Dicionário para guardar as pontuações de recomendação
    scores = {}

    # Para cada filme que o usuário avaliou...
    for movie in rated_movies:
        # Pega os filmes similares a ele
        similar_movies = item_similarity_df[movie].drop(movie)

        # Para cada filme similar, calcula a pontuação de recomendação
        for similar_movie, similarity_score in similar_movies.items():
            # A pontuação é o produto da similaridade pela nota que o usuário deu ao filme original
            predicted_rating = similarity_score * user_ratings[movie]

            # Adiciona a pontuação ao score total do filme similar
            scores[similar_movie] = scores.get(similar_movie, 0) + predicted_rating

    # Filtrar filmes que o usuário já viu
    unseen_movies = {movie: score for movie, score in scores.items() if user_ratings[movie] == 0}

    # Converter para uma Série Pandas e ordenar
    recommendations_df = pd.Series(unseen_movies).sort_values(ascending=False)

    return recommendations_df.head(num_recommendations)


# --- Testando o sistema de recomendação baseado em itens ---
# Usando o mesmo usuário para comparar os resultados depois
print(f"\n--- Gerando 5 recomendações para o usuário ID: {test_user_id} (Baseado em Itens) ---")
item_based_recommendations = get_item_recommendations(test_user_id, num_recommendations=5)

print(item_based_recommendations)

Calculando a similaridade entre filmes...
Matriz de Similaridade entre Filmes (amostra):
title                          $1,000,000 Duck (1971)  'Night Mother (1986)  \
title                                                                         
$1,000,000 Duck (1971)                       1.000000              0.079344   
'Night Mother (1986)                         0.079344              1.000000   
'Til There Was You (1997)                    0.026626              0.065492   
'burbs, The (1989)                           0.066406              0.098002   
...And Justice for All (1979)                0.080384              0.119003   

title                          'Til There Was You (1997)  'burbs, The (1989)  \
title                                                                          
$1,000,000 Duck (1971)                          0.026626            0.066406   
'Night Mother (1986)                            0.065492            0.098002   
'Til There Was You (1997)            

In [ ]:
# Etapa 5: Avaliando o Desempenho dos Modelos

from sklearn.metrics import mean_absolute_error, mean_squared_error
import numpy as np

# Listas para armazenar as avaliações reais e as previstas
actuals = []
predictions_user_based = []
predictions_item_based = []

# Média global de notas para usar em casos onde a previsão não é possível
global_mean = train_data['rating'].mean()

# --- Loop de Avaliação ---
# Itera sobre o conjunto de teste para prever cada avaliação
print("Iniciando a avaliação dos modelos... Isso pode levar alguns minutos.")

# Usaremos uma amostra menor do conjunto de teste para a avaliação ser mais rápida.
# Se quiser uma avaliação completa (mais lenta), remova o .head(1000)
test_sample = test_data.head(2000)

for _, row in test_sample.iterrows():
    user_id = row['userId']
    movie_title = row['title']
    actual_rating = row['rating']

    # --- Previsão com o modelo User-Based ---
    # Verificar se o usuário e o filme existem na matriz de treino
    if user_id in user_similarity_df.index and movie_title in user_item_matrix.columns:
        similar_users = user_similarity_df[user_id].sort_values(ascending=False)[1:11]
        similar_user_ids = similar_users.index

        # Avaliações do filme alvo feitas pelos usuários similares
        ratings_from_similars = user_item_matrix.loc[similar_user_ids, movie_title]

        # Ponderar as avaliações pela similaridade
        weighted_sum = np.dot(ratings_from_similars, similar_users)
        sum_of_weights = similar_users.sum()

        predicted_rating_user = weighted_sum / sum_of_weights if sum_of_weights > 0 else global_mean
    else:
        predicted_rating_user = global_mean # Cold start: usuário ou filme novo

    # --- Previsão com o modelo Item-Based ---
    if movie_title in item_similarity_df.index and user_id in item_user_matrix.columns:
        user_ratings_for_items = item_user_matrix[user_id]
        rated_items = user_ratings_for_items[user_ratings_for_items > 0]

        similar_items = item_similarity_df[movie_title].loc[rated_items.index].sort_values(ascending=False)[1:11]

        weighted_sum = np.dot(similar_items, rated_items.loc[similar_items.index])
        sum_of_weights = similar_items.sum()

        predicted_rating_item = weighted_sum / sum_of_weights if sum_of_weights > 0 else global_mean
    else:
        predicted_rating_item = global_mean # Cold start

    # Adicionar resultados às listas
    actuals.append(actual_rating)
    predictions_user_based.append(predicted_rating_user)
    predictions_item_based.append(predicted_rating_item)

print("Avaliação concluída.")

# --- Calcular as métricas ---
# User-Based
mae_user = mean_absolute_error(actuals, predictions_user_based)
rmse_user = np.sqrt(mean_squared_error(actuals, predictions_user_based))

# Item-Based
mae_item = mean_absolute_error(actuals, predictions_item_based)
rmse_item = np.sqrt(mean_squared_error(actuals, predictions_item_based))


# --- Exibir os resultados ---
print("\n--- Resultados da Avaliação ---")
print(f"Modelo de Filtragem Colaborativa Baseado em USUÁRIOS:")
print(f"  - MAE: {mae_user:.4f}")
print(f"  - RMSE: {rmse_user:.4f}")

print(f"\nModelo de Filtragem Colaborativa Baseado em ITENS:")
print(f"  - MAE: {mae_item:.4f}")
print(f"  - RMSE: {rmse_item:.4f}")

Iniciando a avaliação dos modelos... Isso pode levar alguns minutos.
Avaliação concluída.

--- Resultados da Avaliação ---
Modelo de Filtragem Colaborativa Baseado em USUÁRIOS:
  - MAE: 2.0770
  - RMSE: 2.3810

Modelo de Filtragem Colaborativa Baseado em ITENS:
  - MAE: 0.7501
  - RMSE: 0.9759
